# Shallow Water Bathymetry <a id="top"></a>
## Visualizing Differences in Depth With Spectral Analysis
<hr>

# Notebook Summary

* Import data from LANDSAT 8 that has been S3 indexed into the Data Cube database
* A bathymetry index is calculated
* Contrast is adjusted to make a more interpretable visualization.
>Citation: [Stumpf, Richard P., Kristine Holderied, and Mark Sinclair. "Determination of water depth with high‐resolution satellite imagery over variable bottom types." Limnology and Oceanography 48.1part2 (2003): 547-556.](https://www.slideshare.net/fernandojeffersonprudencioparedes/stumpf-et-al-2003)
<hr>

# Algorithmic process  

* [Import dependencies and connect to the data cube](#import)
* [Choose platform and product](#plat_prod)
* [Define spatial extents that fall within the maximum extents of the indexed S3 tile](#define_extents) (selecting too much can make the acquisition process slow)
* [Retrieve the data](#retrieve_data)
* [Calculate bathymetry and NDWI indices](#bathymetry)
* [Export the unmasked dataset with NDWI and bathymetry columns to GeoTIFF](#export_unmasked)
* [Clean mask using the quality column and NDWI](#mask)
* [Make a visualization function to view the bathymetry index over the specified region](#vis_func)
* [Examine the bathymetry visualization](#bath_vis)
* [Examine the bathymetry visualization with adjusted contrast](#bath_vis_better)
* [Export the masked dataset with NDWI and bathymetry columns to GeoTIFF](#export_masked)


<hr>

# How It Works

Bathymetry is the measurement of depth in bodies of water(Oceans, Seas or Lakes).  This notebook illustrates a technique for deriving depth of shallow water areas using purely optical features from Landsat Collection 1 imagery and draws heavily from the publication [Determination of water depth with high-resolution satelite imagery over variable bottom types](https://www.slideshare.net/fernandojeffersonprudencioparedes/stumpf-et-al-2003).  

<br>

**Bathymetry Index**  
  
This bathymetry index uses optical `green` and `blue` values on a logarithmic scale with two tunable coefficients `m0` and `m1`.
  

$$ BATH =  m_0*\frac{ln(blue)}{ln(green)} -m_1$$  

Where: 
- `m0` is a tunable scaling factor to tune the ratio to depth <br>
- `m1` is the offset for a depth of 0 meters.

<br>
<div class="alert-info"><br>
<b>Note: </b> that for our purposes, $m_0$ and $m_1$ are equal to <b>1</b> and <b>0</b> respectively, since we cannot determine the baseline nor the offset from spectral reflectance alone. This effectively simplifies the formula to: $$\frac{ln(blue)}{ln(green)}$$
<br>

</div>


#### Bathymetry Index Function

In [1]:
import numpy as np
import pandas as pd

def bathymetry_index(df, m0 = 1, m1 = 0):
    return m0*(np.log(df.blue)/np.log(df.green))+m1

<hr>

## <a id="import">Import Dependencies and Connect to the Data Cube</a>  [&#9652;](#top)

In [2]:
% matplotlib inline
import datacube
dc = datacube.Datacube()

/home/localuser/Datacube/datacube_env/lib/python3.5/site-packages/psycopg2-2.7.5-py3.5-linux-x86_64.egg/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


<hr>

## <a id="plat_prod">Select the Product and Platform</a>  [&#9652;](#top)

In [3]:
#List the products available on this server/device
dc.list_products()

,name,description,time,product_type,instrument,lon,lat,format,platform,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,
0,ls7_collections_sr_scene,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,ETM,None,None,GeoTiff,LANDSAT_7,NaN,NaN,NaN,NaN
1,ls7_ledaps_vietnam,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,ETM,None,None,NetCDF,LANDSAT_7,EPSG:4326,"[-0.000269494585236, 0.000269494585236]","[0.943231048326, 0.943231048326]","(latitude, longitude)"
2,ls7_ledaps_vietnam_sample,Sample subset of ls7_ledaps_vietnam created fo...,None,LEDAPS,ETM,None,None,NetCDF,LANDSAT_7,EPSG:4326,"[-0.000269494585236, 0.000269494585236]","[0.943231048326, 0.943231048326]","(latitude, longitude)"


In [4]:
#create a list of the desired platforms
platform = "LANDSAT_8"
product = "ls8_level1_usgs"

<hr>

## <a id="define_extents">Define and Display the Region to Be Examined</a>  [&#9652;](#top)

### Region bounds

In [5]:
lat_subsect = (-31.7, -32.2)
lon_subsect = (152.4, 152.9)

In [6]:
print('''
Latitude:\t{0}\t\tRange:\t{2} degrees
Longitude:\t{1}\t\tRange:\t{3} degrees
'''.format(lat_subsect,
           lon_subsect,
           max(lat_subsect)-min(lat_subsect),
           max(lon_subsect)-min(lon_subsect)))


Latitude:	(-31.7, -32.2)		Range:	0.5000000000000036 degrees
Longitude:	(152.4, 152.9)		Range:	0.5 degrees



### Display

In [7]:
from utils.data_cube_utilities.dc_display_map import display_map      
display_map(latitude = lat_subsect,longitude = lon_subsect)

<hr>

## <a id="retrieve_data">Retrieve the Data</a>  [&#9652;](#top)

#### Load and integrate datasets

In [8]:
%%time
ds = dc.load(lat = lat_subsect,
             lon = lon_subsect,
             platform = platform,
             product = product,
             output_crs = "EPSG:32756",
             measurements = ["red","blue","green","nir","quality"],
             resolution = (-30,30))

CPU times: user 20 ms, sys: 12 ms, total: 32 ms
Wall time: 48.1 ms


In [9]:
ds

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*

#### Preview the Data

In [10]:
import matplotlib.pyplot as plt

def rgb(dataset, at_index = 0, bands = ['red', 'green', 'blue'],
        max_possible = 15000):
    
    ### Dataset to RGB Format, needs float values between 0-1 
    rgb = np.stack([dataset[bands[0]],
                    dataset[bands[1]],
                    dataset[bands[2]]], axis = -1).astype(np.int16)
    
    # Clamp to range 0:max_possible
    rgb[rgb<0] = 0    
    rgb[rgb > max_possible] = max_possible
    
    rgb = rgb.astype(float)
    rgb /= np.max(rgb)
    
    # Plot
    plt.figure(figsize=(15,15))   
    plt.imshow(rgb[at_index])

In [11]:
rgb(ds, at_index=6)

KeyError: 'red'

<hr>

## <a id="bathymetry">Calculate the Bathymetry and NDWI Indices</a>  [&#9652;](#top)
> * Bathymetry function located at top of notebook

In [ ]:
# Create Bathemtry Index column
ds["bathymetry"] = bathymetry_index(ds)

In [ ]:
def NDWI_index(ds):
    return (ds.green - ds.nir)/(ds.green + ds.nir)

In [ ]:
ds["ndwi"] = NDWI_index(ds)

<hr>

#### Preview Combined Dataset

In [ ]:
ds

<hr>

## <a id="export_unmasked">Export Unmasked GeoTIFF</a>  [&#9652;](#top)

In [ ]:
import time
def time_to_string(t):
    return time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime(t.astype(int)/1000000000))

In [ ]:
from utils.data_cube_utilities import dc_utilities

def export_slice_to_geotiff(ds, path):
    dc_utilities.write_geotiff_from_xr(path,
                                       ds.astype(np.float32),
                                       list(ds.data_vars.keys()),
                                       vertical_dim = "y",
                                       horizontal_dim = "x",
                                       crs="EPSG:32756")

In [ ]:
def export_xarray_to_geotiff(ds, path):
    for t in ds.time:
        time_slice_xarray = ds.sel(time = t)
        export_slice_to_geotiff(time_slice_xarray,
                                path + "_" + time_to_string(t) + ".tif")

In [ ]:
export_xarray_to_geotiff(ds, "geotiffs/landsat8/unmasked/unmasked")

<hr>

## <a id="mask">Mask the Dataset using the Quality column and NDWI</a>  [&#9652;](#top)

>Notice that the quality column is not the usual bitmask values we are familiar with. <br> More information on `Landsat 8 OLI/ OLI-TIRS Level-1` can be found [here](https://landsat.usgs.gov/collectionqualityband)

In [ ]:
# preview values
np.unique(ds["quality"])

In [ ]:
#make a new function for masking this type of data since we do not have a pixel_qa column
# We are only checking for "clear" but will include the other cover_types
# in case this function is repurposed for other cover_types
def ls8_quality_unpack(data_array, cover_type):  
    
    land_cover_endcoding = dict(fill         =[1] , 
                                terrain_occ  =[2, 2722],
                                clear        =[2720, 2724, 2728, 2732],
                                cloud        =[2800, 2804, 2808, 2812, 6896, 6900, 6904, 6908],
                                rad_sat_1_2  =[2724, 2756, 2804, 2980, 3012, 3748, 3780, 6820, 6852, 6900, 7076, 7108, 7844, 7876],
                                rad_sat_3_4  =[2728, 2760, 2808, 2984, 3016, 3752, 3784, 6824, 6856, 6904, 7080, 7112, 7848, 7880],
                                rad_sat_5_pls=[2732, 2764, 2812, 2988, 3020, 3756, 3788, 6828, 6860, 6908, 7084, 7116, 7852, 7884],
                                low_conf_cl  =[2752, 2722, 2724, 2728, 2732, 2976, 2980, 2984, 2988, 3744, 3748, 3752, 3756, 6816, 6820, 6824, 6828, 7072, 7076, 7080, 7084, 7840, 7844, 7848, 7852],
                                med_conf_cl  =[2752, 2756, 2760, 2764, 3008, 3012, 3016, 3020, 3776, 3780, 3784, 3788, 6848, 6852, 6856, 6860, 7104, 7108, 7112, 7116, 7872, 7876, 7880, 7884],
                                high_conf_cl =[2800, 2804, 2808, 2812, 6896, 6900, 6904, 6908],
                                low_conf_cir =[2720, 2722, 2724, 2728, 2732, 2752, 2756, 2760, 2764, 2800, 2804, 2808, 2812, 2976, 2980, 2984, 2988, 3008, 3012, 3016, 3020, 3744, 3748, 3752, 3756, 3780, 3784, 3788],
                                high_conf_cir=[6816, 6820, 6824, 6828, 6848, 6852, 6856, 6860, 6896, 6900, 6904, 6908, 7072, 7076, 7080, 7084, 7104, 7108, 7112, 7116, 7840, 7844, 7848, 7852, 7872, 7876, 7880, 7884],
                                high_snow_ice=[3744, 3748, 3752, 3756, 3776, 3780, 3784, 3788, 7840, 7844, 7848, 7852, 7872, 7876, 7880, 7884],
                                high_cl_shdw =[2976, 2980, 2984, 2988, 3008, 3012, 3016, 3020, 7072, 7076, 7080, 7084, 7104, 7108, 7112, 7116]
                               )
    return np.isin(data_array.quality.values, land_cover_endcoding[cover_type])

#### Use NDWI to Mask Out Land
> The threshold can be tuned if need be to better fit the RGB image above. <br>
> Unfortunately our existing WOFS algorithm is designed to work with Surface Reflectance (SR) and does not work with this data yet but with a few modifications it could be made to do so.  We will approximate the WOFs mask with `NDWI` for now.

In [ ]:
# Tunable threshold for masking the land out
threshold = .05

water = (ds.ndwi>threshold).values

In [ ]:
#preview one time slice to determine the effectiveness of the NDWI masking
rgb(ds.where(water), at_index=6)

In [ ]:
clear_xarray  = ls8_quality_unpack(ds, "clear")

In [ ]:
full_mask = np.logical_and(clear_xarray, water)

ds = ds.where(full_mask)

<hr>

## <a id="vis_func">Create a Visualization Function</a>  [&#9652;](#top)

#### Visualize the distribution of the bathymetry index for the water pixels

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=[15,5])

#Visualize the distribution of the remaining data
sns.boxplot(ds['bathymetry'])

> <b>Interpretation: </b> We can see that most of the values fall within a very short range.  We can scale our plot's cmap limits to fit the specific quantile ranges for the bathymetry index so we can achieve better contrast from our plots.

In [ ]:
#set the quantile range in either direction from the median value
def get_quantile_range(col, quantile_range = .25):
    low = ds[col].quantile(.5 - quantile_range,["time","y","x"]).values
    high = ds[col].quantile(.5 + quantile_range,["time","y","x"]).values
    return low,high

In [ ]:
#Custom function for a color mapping object
from matplotlib.colors import LinearSegmentedColormap

def custom_color_mapper(name = "custom", val_range = (1.96,1.96), colors = "RdGnBu"):
    custom_cmap = LinearSegmentedColormap.from_list(name,colors=colors)
    
    min, max = val_range
    step = max/10.0
    Z = [min,0],[0,max]
    levels = np.arange(min,max+step,step)
    cust_map = plt.contourf(Z, 100, cmap=custom_cmap)
    plt.clf()
    return cust_map.cmap

In [ ]:
def mean_value_visual(ds, col, figsize = [15,15], cmap = "GnBu", low=None, high=None):
    if low is None: low = np.min(ds[col]).values
    if high is None: high = np.max(ds[col]).values
    ds.reduce(np.nanmean,dim=["time"])[col].plot.imshow(figsize = figsize, cmap=cmap,  vmin=low, vmax=high)

<hr>

## <a id="bath_vis">Visualize the Bathymetry</a>  [&#9652;](#top)

In [ ]:
mean_value_visual(ds, "bathymetry", cmap="GnBu")

<hr>

## <a id="bath_vis_better">Visualize the Bathymetry With Adjusted Contrast</a>  [&#9652;](#top)

> If we clamp the range of the plot using different quantile ranges we can see relative differences in higher contrast.

In [ ]:
# create range using the 10th and 90th quantile
low, high = get_quantile_range("bathymetry", .40)


custom = custom_color_mapper(val_range=(low,high),
                             colors=["darkred","red","orange","yellow","green","blue","darkblue","black"])

mean_value_visual(ds, "bathymetry", cmap=custom, low=low, high=high)

In [ ]:
# create range using the 5th and 95th quantile
low, high = get_quantile_range("bathymetry", .45)


custom = custom_color_mapper(val_range=(low,high),
                             colors=["darkred","red","orange","yellow","green","blue","darkblue","black"])

mean_value_visual(ds, "bathymetry", cmap = custom, low=low, high = high)

In [ ]:
# create range using the 2nd and 98th quantile
low, high = get_quantile_range("bathymetry", .48)


custom = custom_color_mapper(val_range=(low,high),
                             colors=["darkred","red","orange","yellow","green","blue","darkblue","black"])

mean_value_visual(ds, "bathymetry", cmap=custom, low=low, high=high)

In [ ]:
# create range using the 1st and 99th quantile
low, high = get_quantile_range("bathymetry", .49)


custom = custom_color_mapper(val_range=(low,high),
                             colors=["darkred","red","orange","yellow","green","blue","darkblue","black"])

mean_value_visual(ds, "bathymetry", cmap=custom, low=low, high=high)

<hr>

## <a id="export_masked">Export the Masked GeoTIFF</a>  [&#9652;](#top)

In [ ]:
export_xarray_to_geotiff(ds, "geotiffs/landsat8/masked/masked")